In [ ]:
# --- CELDA ÚNICA: Aplicación Completa (VERSIÓN INTEGRADA) ---
# Esta versión combina la app de 8 pestañas con la nueva lógica
# de coeficientes (Cantidad, Pn Total, Ku, Ks).
# CORREGIDA PARA BINDER: Se eliminó la importación de 'google.colab'

print("Cargando librerías estándar...")

# --- Importar las bibliotecas necesarias ---
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, SVG
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# ¡LÍNEA CORREGIDA! La importación 'from google.colab import files' se eliminó.
import io
import openpyxl # Requerido por pandas para Excel
import json # Para Guardar/Cargar
from base64 import b64encode # Para descargas
import warnings # Para suprimir advertencias

# Suprimir advertencias menores
warnings.filterwarnings('ignore', category=FutureWarning)


# --- 1. ESTADO GLOBAL DE LA APLICACIÓN ---

# Estilo para las descripciones de los widgets (label)
style_descripcion = {'description_width': '200px'}
layout_input = widgets.Layout(width='95%')

# Modos de operación (columnas del balance)
modos_operacion = ["Navegación", "Maniobra", "Puerto", "Carga/Descarga", "Emergencia"]

# Opciones de Barras para el Diagrama
barras_opciones = ["N/A", "MSB-A", "MSB-B", "EMSB", "220V (T1)"]

# --- BASE DE DATOS DE CABLES MEJORADA (CON AMPACIDAD) ---
CABLE_DATABASE = {
    # Sección (mm²): {'R_ohm_km': ..., 'X_ohm_km': ..., 'Ampacidad_A': ...}
    "16 mm²":  {'R_ohm_km': 1.21,  'X_ohm_km': 0.082, 'Ampacidad_A': 76},
    "25 mm²":  {'R_ohm_km': 0.78,  'X_ohm_km': 0.081, 'Ampacidad_A': 101},
    "35 mm²":  {'R_ohm_km': 0.554, 'X_ohm_km': 0.080, 'Ampacidad_A': 129},
    "50 mm²":  {'R_ohm_km': 0.386, 'X_ohm_km': 0.079, 'Ampacidad_A': 158},
    "70 mm²":  {'R_ohm_km': 0.272, 'X_ohm_km': 0.078, 'Ampacidad_A': 201},
    "95 mm²":  {'R_ohm_km': 0.196, 'X_ohm_km': 0.077, 'Ampacidad_A': 250},
    "120 mm²": {'R_ohm_km': 0.154, 'X_ohm_km': 0.076, 'Ampacidad_A': 292},
    "150 mm²": {'R_ohm_km': 0.124, 'X_ohm_km': 0.076, 'Ampacidad_A': 337},
}

# --- Base de Datos de Generadores ---
GEN_DATABASE = [
    # (Fabricante, Modelo, Potencia_kW_60Hz, Potencia_kW_50Hz)
    ("Caterpillar", "C7.1", 175, 150),
    ("Caterpillar", "C9.3", 300, 250),
    ("Caterpillar", "C18", 550, 450),
    ("Caterpillar", "C18 ACERT", 600, 500),
    ("Caterpillar", "C32", 940, 800),
    ("Caterpillar", "3508C", 800, 680),
    ("Caterpillar", "3512C", 1360, 1150),
    ("Caterpillar", "3516C", 2000, 1700),
    ("Caterpillar", "3516E", 2550, 2200),
    ("Cummins", "QSK19-DM", 560, 460),
    ("Cummins", "QSK38-DM", 1200, 1000),
    ("Cummins", "QSK50-DM", 1600, 1350),
    ("Cummins", "QSK60-DM", 2000, 1800),
    ("Cummins", "QSK95-DM", 3200, 2800),
    ("Volvo Penta", "D9 MG", 260, 215),
    ("Volvo Penta", "D13 MG", 400, 330),
    ("Volvo Penta", "D16 MG", 550, 460),
    ("MAN", "D2868 LE421", 500, 420),
    ("MAN", "D2862 LE431", 750, 630),
    ("MAN", "D2862 LE441", 880, 740),
]

# Base de Datos de Generadores de Emergencia
EMERGENCY_GEN_DATABASE = [
    ("Caterpillar", "C4.4", 80, 65),
    ("Caterpillar", "C4.4 ACERT", 118, 100),
    ("Caterpillar", "C7.1 ACERT", 160, 135),
    ("Cummins", "QSB7-DM", 120, 100),
    ("Cummins", "QSL9-DM", 200, 170),
    ("Volvo Penta", "D5A T", 85, 70),
    ("Volvo Penta", "D8 MG", 180, 150),
]


# Almacenará los resultados clave para pasarlos a otras pestañas
global_app_state = {
    "max_kw": 0, "req_kw_n1_3gen": 0, "req_kw_n1_2gen": 0,
    "emerg_kw": 0, "emerg_kva": 0, "puerto_kw": 0, "puerto_kva": 0,
    "largest_motor_kw": 0, "largest_motor_cos_phi": 0,
    "selected_gen_kw": 0, "selected_gen_kva": 0, "selected_emerg_gen_kw": 0,
    "ultimo_diagrama_svg": ""
}

# --- 2. LÓGICA DE GESTIÓN DE CONSUMIDORES (NUEVA LÓGICA) ---

def get_consumidor_default(nombre, cantidad, pn_kw_total, cos_phi, esencial, barra, modos_ku_ks):
    """
    Crea una estructura de diccionario para un consumidor
    con valores de Ku y Ks para cada modo.
    """
    consumidor = {
        'nombre': nombre,
        'cantidad': cantidad,
        'pn_kw_total': pn_kw_total, # Potencia TOTAL del grupo
        'cos_phi': cos_phi,
        'esencial': esencial,
        'barra': barra,
        'modos': {}
    }
    for modo, (ku, ks) in modos_ku_ks.items():
        consumidor['modos'][modo] = {'ku': ku, 'ks': ks}
    return consumidor

def get_lista_std():
    """
    Retorna una lista de diccionarios de consumidores estándar
    CONVERTIDA a la nueva lógica (Cantidad, Pn Total, Ku, Ks).
    """
    lista = [
        # Generadores (Pn=0 para no sumar carga)
        get_consumidor_default("G-1 (Generador Principal)", 1, 0, 0.8, False, "MSB-A",
            {"Navegación": (0, 0), "Maniobra": (0, 0), "Puerto": (0, 0), "Carga/Descarga": (0, 0), "Emergencia": (0, 0)}),
        get_consumidor_default("G-2 (Generador Principal)", 1, 0, 0.8, False, "MSB-A",
            {"Navegación": (0, 0), "Maniobra": (0, 0), "Puerto": (0, 0), "Carga/Descarga": (0, 0), "Emergencia": (0, 0)}),
        get_consumidor_default("G-3 (Generador Principal)", 1, 0, 0.8, False, "MSB-B",
            {"Navegación": (0, 0), "Maniobra": (0, 0), "Puerto": (0, 0), "Carga/Descarga": (0, 0), "Emergencia": (0, 0)}),
        get_consumidor_default("G-E (Generador Emergencia)", 1, 0, 0.8, True, "EMSB",
            {"Navegación": (0, 0), "Maniobra": (0, 0), "Puerto": (0, 0), "Carga/Descarga": (0, 0), "Emergencia": (0, 0)}),

        # Consumidores (Pn es Pn_TOTAL = Pn_individual * cantidad)
        # Pn_ind=30.0, Cant=2 -> Pn_Total=60.0
        get_consumidor_default("Bombas Serv. General (x2)", 2, 60.0, 0.85, True, "MSB-A",
            {"Navegación": (0.8, 0.5), "Maniobra": (0.5, 1.0), "Puerto": (0.2, 0.5), "Carga/Descarga": (0.5, 1.0), "Emergencia": (1.0, 0.5)}),
        # Pn_ind=55.0, Cant=2 -> Pn_Total=110.0
        get_consumidor_default("Bombas Lastre (x2)", 2, 110.0, 0.85, False, "MSB-B",
            {"Navegación": (0.0, 0.0), "Maniobra": (0.5, 1.0), "Puerto": (0.0, 0.0), "Carga/Descarga": (0.8, 1.0), "Emergencia": (0.0, 0.0)}),
        # Pn_ind=280.0, Cant=1 -> Pn_Total=280.0
        get_consumidor_default("Bow Thruster", 1, 280.0, 0.88, False, "MSB-A",
            {"Navegación": (0.0, 0.0), "Maniobra": (0.4, 1.0), "Puerto": (0.0, 0.0), "Carga/Descarga": (0.0, 0.0), "Emergencia": (0.0, 0.0)}),
        # Pn_ind=25.0, Cant=2 -> Pn_Total=50.0
        get_consumidor_default("Compresores Aire Arranque (x2)", 2, 50.0, 0.80, True, "MSB-A",
            {"Navegación": (0.3, 0.5), "Maniobra": (0.5, 1.0), "Puerto": (0.1, 0.5), "Carga/Descarga": (0.1, 0.5), "Emergencia": (0.0, 0.0)}),
        # Pn_ind=11.0, Cant=2 -> Pn_Total=22.0
        get_consumidor_default("Bomba Gobierno (x2)", 2, 22.0, 0.85, True, "EMSB",
            {"Navegación": (1.0, 0.5), "Maniobra": (1.0, 1.0), "Puerto": (0.0, 0.0), "Carga/Descarga": (0.0, 0.0), "Emergencia": (1.0, 0.5)}),
        # Pn_ind=22.0, Cant=2 -> Pn_Total=44.0
        get_consumidor_default("Ventiladores Sala Máquinas (x2)", 2, 44.0, 0.85, True, "MSB-B",
            {"Navegación": (1.0, 1.0), "Maniobra": (1.0, 1.0), "Puerto": (0.5, 0.5), "Carga/Descarga": (0.8, 1.0), "Emergencia": (1.0, 0.5)}),
        # Pn_ind=15.0, Cant=4 -> Pn_Total=60.0
        get_consumidor_default("Ventiladores Bodega (x4)", 4, 60.0, 0.85, False, "MSB-B",
            {"Navegación": (0.0, 0.0), "Maniobra": (0.0, 0.0), "Puerto": (0.0, 0.0), "Carga/Descarga": (0.9, 1.0), "Emergencia": (0.0, 0.0)}),
        # Pn_ind=75.0, Cant=1 -> Pn_Total=75.0 (Carga de trafo)
        get_consumidor_default("Transformador 440/220V (T1)", 1, 75.0, 0.98, False, "MSB-B",
            {"Navegación": (0.6, 1.0), "Maniobra": (0.7, 1.0), "Puerto": (0.5, 1.0), "Carga/Descarga": (0.8, 1.0), "Emergencia": (0.3, 1.0)}),
        # Pn_ind=40.0, Cant=1 -> Pn_Total=40.0
        get_consumidor_default("Iluminación (General)", 1, 40.0, 0.95, True, "220V (T1)",
            {"Navegación": (0.8, 1.0), "Maniobra": (0.9, 1.0), "Puerto": (0.6, 1.0), "Carga/Descarga": (1.0, 1.0), "Emergencia": (0.3, 1.0)}),
        # Pn_ind=60.0, Cant=1 -> Pn_Total=60.0
        get_consumidor_default("Servicios Alojamiento (HVAC, Galley)", 1, 60.0, 0.90, False, "220V (T1)",
            {"Navegación": (0.7, 1.0), "Maniobra": (0.6, 1.0), "Puerto": (0.5, 1.0), "Carga/Descarga": (0.7, 1.0), "Emergencia": (0.0, 0.0)}),
        # Pn_ind=5.0, Cant=1 -> Pn_Total=5.0
        get_consumidor_default("Equipos Navegación (Puente)", 1, 5.0, 0.98, True, "EMSB",
            {"Navegación": (1.0, 1.0), "Maniobra": (1.0, 1.0), "Puerto": (0.5, 1.0), "Carga/Descarga": (1.0, 1.0), "Emergencia": (1.0, 1.0)}),
        # Pn_ind=30.0, Cant=1 -> Pn_Total=30.0
        get_consumidor_default("Bomba Incendio Emergencia", 1, 30.0, 0.85, True, "EMSB",
            {"Navegación": (0.0, 0.0), "Maniobra": (0.0, 0.0), "Puerto": (0.0, 0.0), "Carga/Descarga": (0.0, 0.0), "Emergencia": (1.0, 1.0)}),
    ]
    return [get_consumidor_dict(c) for c in lista]

def get_consumidor_dict(c_base):
    """
    Toma la estructura base y la convierte en el diccionario
    completo que usa la aplicación (incluyendo widgets).
    --- REFACTORIZADO PARA NUEVA LÓGICA ---
    """
    consumidor = {
        'id': f"consumidor_{np.random.randint(1000, 9999)}",
        'nombre_w': widgets.Text(value=c_base['nombre']),
        'qty_w': widgets.IntText(value=c_base['cantidad'], layout=widgets.Layout(width='95%')),
        'pn_w': widgets.FloatText(value=c_base['pn_kw_total'], layout=widgets.Layout(width='95%')), # Pn TOTAL
        'cos_w': widgets.FloatText(value=c_base['cos_phi'], layout=widgets.Layout(width='95%')),
        'esencial_w': widgets.Checkbox(value=c_base['esencial'], description="Esencial", style={'description_width': 'initial'}),
        'barra_w': widgets.Dropdown(options=barras_opciones, value=c_base['barra']),
        'modos_widgets': {}
    }

    # Crear widgets para cada modo
    for modo in modos_operacion:
        ku_val = c_base['modos'][modo]['ku']
        ks_val = c_base['modos'][modo]['ks']

        consumidor['modos_widgets'][modo] = {
            'ku': widgets.FloatText(value=ku_val, layout=widgets.Layout(width='120px'), description="Ku (Uso):", style={'description_width': '70px'}),
            'ks': widgets.FloatText(value=ks_val, layout=widgets.Layout(width='120px'), description="Ks (Simul):", style={'description_width': '70px'})
        }
    return consumidor


# Almacenará la lista de todos los consumidores que agreguemos.
global_consumidores = get_lista_std()


# --- 2. WIDGETS DE ENTRADA (PESTAÑA 1: Configuración) ---

# --- Sección para agregar un nuevo consumidor (NUEVA LÓGICA) ---
nuevo_consumidor_titulo = widgets.HTML("<h3>Agregar Nuevo Consumidor (en blanco)</h3>")
nuevo_nombre = widgets.Text(description="Nombre:", style=style_descripcion, layout=layout_input)

# Observer para la Cantidad: Actualiza los Ks por defecto
def on_nuevo_cantidad_change(change):
    new_qty = change['new']
    if new_qty > 0:
        new_ks = 1.0 / new_qty
        # Actualizar todos los Ks del formulario de nuevo consumidor
        for modo in modos_operacion:
             nuevo_modos_w[modo]['ks'].value = new_ks
    elif new_qty == 1:
        for modo in modos_operacion:
             nuevo_modos_w[modo]['ks'].value = 1.0

nuevo_cantidad = widgets.IntText(description="Cantidad (n):", value=1, style=style_descripcion, layout=layout_input)
nuevo_cantidad.observe(on_nuevo_cantidad_change, names='value')

nuevo_pn_total = widgets.FloatText(description="Pn TOTAL (kW):", value=10.0, style=style_descripcion, layout=layout_input, tooltip="Potencia total instalada del grupo (ej. 2 bombas de 30kW -> Pn TOTAL = 60kW)")
nuevo_cos_phi = widgets.FloatText(description="Cos φ:", value=0.85, style=style_descripcion, layout=layout_input)
nuevo_esencial = widgets.Checkbox(description="Esencial (SOLAS)", value=False, style=style_descripcion)
nuevo_barra = widgets.Dropdown(options=barras_opciones, value="N/A", description="Barra (SLD):", style=style_descripcion, layout=layout_input)

# Inputs para los modos del nuevo consumidor
nuevo_modos_w = {}
items_modos = []
for modo in modos_operacion:
    ku = widgets.FloatText(value=0.0, description="Ku (Uso):", layout=widgets.Layout(width='120px'), style={'description_width': '70px'})
    ks = widgets.FloatText(value=1.0, description="Ks (Simul):", layout=widgets.Layout(width='120px'), style={'description_width': '70px'}) # Default 1.0 para n=1
    nuevo_modos_w[modo] = {'ku': ku, 'ks': ks}
    items_modos.append(widgets.VBox([widgets.HTML(f"<b>{modo}</b>"), widgets.HBox([ku, ks])]))

layout_modos_accordion = widgets.Accordion(
    children=[widgets.HBox(items_modos, layout=widgets.Layout(width='100%', justify_content='space-around'))],
    selected_index=None # Colapsado por defecto
)
layout_modos_accordion.set_title(0, "Ver/Ocultar Coeficientes (Ku y Ks)")

boton_agregar = widgets.Button(description="Agregar Consumidor", button_style='success', icon='plus', layout=widgets.Layout(margin='10px 0'))

# --- Sección para mostrar la lista de consumidores agregados ---
lista_consumidores_titulo = widgets.HTML("<h2>1. Lista de Consumidores</h2>")
boton_cargar_std = widgets.Button(description="Cargar Lista Estándar", button_style='info', icon='list', layout=widgets.Layout(margin='5px'))
boton_limpiar_lista = widgets.Button(description="Limpiar Toda la Lista", button_style='danger', icon='trash', layout=widgets.Layout(margin='5px'))
boton_guardar_json = widgets.Button(description="Guardar Config.", button_style='primary', icon='save', layout=widgets.Layout(margin='5px'))
upload_json = widgets.FileUpload(accept='.json', description="Cargar Config.", multiple=False, button_style='warning', layout=widgets.Layout(width='300px'))
json_output = widgets.Output() # Para mensajes de Guardar/Cargar/Actualizar
boton_actualizar_calculos = widgets.Button(
    description="Actualizar Balance y Cálculos", button_style='success', icon='refresh',
    layout=widgets.Layout(margin='10px 0', width='95%')
)
lista_consumidores_output = widgets.VBox([])

# Layout completo de la Pestaña 1
layout_tab1 = widgets.VBox([
    lista_consumidores_titulo,
    widgets.HBox([boton_cargar_std, boton_limpiar_lista]),
    widgets.HBox([boton_guardar_json, upload_json]),
    json_output,
    widgets.HTML("<hr>"),
    boton_actualizar_calculos,
    widgets.HTML("<p><i>Edite los valores (Ku/Ks) en los acordeones y luego presione 'Actualizar'.</i></p>"),
    lista_consumidores_output,
    widgets.HTML("<hr>"),
    nuevo_consumidor_titulo,
    nuevo_nombre,
    widgets.HBox([nuevo_cantidad, nuevo_pn_total], layout=widgets.Layout(justify_content='space-between')), # Cantidad y Pn Total
    nuevo_cos_phi,
    nuevo_esencial,
    nuevo_barra,
    widgets.HTML("<b>Coeficientes (Ku y Ks) para cada modo:</b>"),
    layout_modos_accordion,
    boton_agregar
])


# --- 3. PESTAÑAS DE SALIDA (SIN CAMBIOS) ---

# --- Pestaña 2: Balance de Cargas (Output) ---
balance_output = widgets.Output()
balance_chart_output = widgets.Output()
boton_exportar_excel = widgets.Button(description="Exportar a Excel", button_style='primary', icon='download', layout=widgets.Layout(margin='10px 0'))
balance_export_output = widgets.Output()

layout_tab2 = widgets.VBox([
    widgets.HTML("<h2>2. Balance de Cargas Calculado</h2>"),
    widgets.HTML("<p>Fórmula: <b>P_activa = Pn_TOTAL * Ku * Ks</b></p>"),
    balance_output,
    widgets.HTML("<hr><h3>Gráfico de Cargas (kW)</h3>"),
    balance_chart_output,
    boton_exportar_excel,
    balance_export_output
])

# --- Pestaña 3: Dimensionamiento (Output y Widgets) ---
sizing_output = widgets.Output()
input_carga_trafo_kva = widgets.FloatText(description="Carga Trafo (kVA):", value=70.0, style=style_descripcion, layout=layout_input)
boton_calc_trafo = widgets.Button(description="Calcular Trafo (115%)", button_style='info')
trafo_output = widgets.Output()
motor_arranque_label = widgets.HTML(value="Motor más grande: <b>---</b>")
motor_lrc_factor = widgets.FloatText(description="Factor LRC (x In):", value=6.5, style=style_descripcion, layout=layout_input, tooltip="Típico: 6.0-8.0")
motor_cos_phi_arranque = widgets.FloatText(description="Cos φ Arranque:", value=0.3, style=style_descripcion, layout=layout_input, tooltip="Típico: 0.2-0.4")
gen_xd_transient = widgets.FloatText(description="Gen. Xd'' (p.u.):", value=0.15, style=style_descripcion, layout=layout_input, tooltip="Reactancia subtransitoria, 0.15-0.20")
boton_verificar_arranque = widgets.Button(description="Verificar Arranque de Motor", button_style='info')
arranque_output = widgets.Output()
label_shore_carga = widgets.HTML(value="Carga en 'Puerto' (Calculada): <b>--- kVA</b>", layout=layout_input)
input_shore_kva = widgets.FloatText(description="Toma de Tierra (kVA):", value=59.0, style=style_descripcion, layout=layout_input)
boton_check_shore = widgets.Button(description="Verificar Toma de Tierra", button_style='info')
shore_output = widgets.Output()

layout_tab3 = widgets.VBox([
    widgets.HTML("<h2>3. Dimensionamiento Generadores (Planta Principal)</h2>"),
    sizing_output,
    widgets.HTML("<hr><h2>4. Verificación de Arranque de Motores</h2>"),
    widgets.VBox([
        motor_arranque_label, motor_lrc_factor, motor_cos_phi_arranque,
        gen_xd_transient, boton_verificar_arranque, arranque_output
    ], layout=widgets.Layout(border='1px solid #ccc', padding='10px', margin='10px 0')),
    widgets.HTML("<hr><h2>5. Dimensionamiento Transformadores</h2>"),
    widgets.VBox([
        input_carga_trafo_kva, boton_calc_trafo, trafo_output
    ], layout=widgets.Layout(border='1px solid #ccc', padding='10px', margin='10px 0')),
    widgets.HTML("<hr><h2>6. Verificación Toma de Tierra (Shore Connection)</h2>"),
    widgets.VBox([
        label_shore_carga, input_shore_kva, boton_check_shore, shore_output
    ], layout=widgets.Layout(border='1px solid #ccc', padding='10px', margin='10px 0'))
])

# --- Pestaña 4: Cálculos Auxiliares ---
volt_drop_titulo = widgets.HTML("<h3>Cálculo de Caída de Tensión (Trifásico AC)</h3>")
volt_drop_nota = widgets.HTML("<p><i>Verifica Caída de Tensión (límite 6%) y Ampacidad (seguridad).</i></p>")
volt_I = widgets.FloatText(description="Corriente (A):", value=50, style=style_descripcion)
volt_L = widgets.FloatText(description="Longitud Cable (m):", value=40, style=style_descripcion)
volt_seccion = widgets.Dropdown(options=list(CABLE_DATABASE.keys()), description="Sección Cable:", value="25 mm²", style=style_descripcion)
volt_cosphi = widgets.FloatText(description="Cos φ de la Carga:", value=0.85, min=0.1, max=1.0, step=0.05, style=style_descripcion)
volt_Vn = widgets.FloatText(description="Tensión Nominal (V):", value=440, style=style_descripcion)
volt_boton = widgets.Button(description="Calcular Caída y Ampacidad", button_style='info')
volt_output = widgets.Output()
short_circuit_titulo = widgets.HTML("<hr><h3>Calculadora de Corriente de Cortocircuito (Icc)</h3>")
short_circuit_nota = widgets.HTML("<p>Calcula la Icc simétrica en bornes del generador (usando datos de Pestaña 6).</p>")
icc_gen_kva_label = widgets.HTML(value="Gen. kVA (de Pestaña 6): <b>--- kVA</b>", layout=layout_input)
icc_gen_xd = widgets.FloatText(description="Gen. Xd'' (p.u.):", value=0.15, style=style_descripcion, layout=layout_input, tooltip="Reactancia subtransitoria, 0.15-0.20")
icc_Vn = widgets.FloatText(description="Tensión (V):", value=440, style=style_descripcion, layout=layout_input)
boton_calc_icc = widgets.Button(description="Calcular Icc (kA)", button_style='info')
icc_output = widgets.Output()

layout_tab4 = widgets.VBox([
    volt_drop_titulo, volt_drop_nota,
    volt_I, volt_L, volt_seccion, volt_cosphi, volt_Vn,
    volt_boton, volt_output,
    short_circuit_titulo, short_circuit_nota,
    widgets.VBox([
        icc_gen_kva_label, icc_gen_xd, icc_Vn, boton_calc_icc, icc_output
    ], layout=widgets.Layout(border='1px solid #ccc', padding='10px', margin='10px 0'))
])

# --- Pestaña 5: Verificación de Requisitos ---
requisitos_output = widgets.Output(layout=widgets.Layout(border='1px solid #ccc', padding='10px'))
layout_tab5 = widgets.VBox([
    widgets.HTML("<h2>5. Verificación de Requisitos (SOLAS N-1)</h2>"),
    requisitos_output
])

# --- Pestaña 6: Selección Generador Principal ---
gen_config_select = widgets.Dropdown(
    options=[('2 Generadores (N-1)', '2gen'), ('3 Generadores (N-1)', '3gen')],
    value='3gen', description='Configuración N-1:', style=style_descripcion
)
gen_freq_select = widgets.Dropdown(options=[60, 50], value=60, description="Frecuencia (Hz):", style=style_descripcion)
gen_margin_slider = widgets.IntSlider(value=15, min=0, max=50, step=5, description="Margen (%):", style=style_descripcion)
gen_req_label = widgets.HTML(value="Potencia Requerida por Generador: <b>--- kW</b>")
gen_buscar_boton = widgets.Button(description="Buscar Modelos", button_style='primary', icon='search')
gen_resultados_select = widgets.Select(options=[], description='Modelos Encontrados:', layout=widgets.Layout(width='95%', height='200px'), style=style_descripcion)
gen_detalle_output = widgets.Output(layout=widgets.Layout(border='1px solid #ccc', padding='10px', width='95%'))

layout_tab6 = widgets.VBox([
    widgets.HTML("<h2>6. Selección Comercial (Generadores Principales)</h2>"),
    widgets.VBox([
        gen_config_select, gen_freq_select, gen_margin_slider,
        gen_req_label, gen_buscar_boton, gen_resultados_select, gen_detalle_output
    ], layout=widgets.Layout(border='1px solid #ccc', padding='10px', margin='10px 0'))
])

# --- Pestaña 7: Planta de Emergencia ---
emerg_load_label = widgets.HTML(value="Carga de Emergencia Calculada: <b>--- kW / --- kVA</b>")
emerg_consumidores_output = widgets.Output(layout=widgets.Layout(border='1px solid #ccc', padding='10px', width='95%', height='150px', overflow_y='auto'))
emerg_source_select = widgets.RadioButtons(options=['Generador', 'Baterías'], value='Generador', description='Fuente Emergencia:', style=style_descripcion)
emerg_gen_freq_select = widgets.Dropdown(options=[60, 50], value=60, description="Frecuencia (Hz):", style=style_descripcion)
emerg_gen_margin_slider = widgets.IntSlider(value=20, min=0, max=50, step=5, description="Margen (%):", style=style_descripcion)
emerg_gen_buscar_boton = widgets.Button(description="Buscar Gen. Emergencia", button_style='primary', icon='search')
emerg_gen_resultados_select = widgets.Select(options=[], description='Modelos Encontrados:', layout=widgets.Layout(width='95%', height='150px'), style=style_descripcion)
emerg_gen_detalle_output = widgets.Output(layout=widgets.Layout(border='1px solid #ccc', padding='10px', width='95%'))
emerg_gen_box = widgets.VBox([
    emerg_gen_freq_select, emerg_gen_margin_slider, emerg_gen_buscar_boton,
    emerg_gen_resultados_select, emerg_gen_detalle_output
])
emerg_bat_horas = widgets.FloatText(description="Autonomía (horas):", value=18.0, style=style_descripcion, layout=layout_input)
emerg_bat_voltaje = widgets.Dropdown(options=[24, 48, 110, 220], value=24, description="Tensión Banco (V):", style=style_descripcion, layout=layout_input)
emerg_bat_calc_boton = widgets.Button(description="Calcular Banco Baterías", button_style='info', icon='calculator')
emerg_bat_output = widgets.Output(layout=widgets.Layout(border='1px solid #ccc', padding='10px', width='95%'))
emerg_bat_box = widgets.VBox(
    [emerg_bat_horas, emerg_bat_voltaje, emerg_bat_calc_boton, emerg_bat_output],
    layout=widgets.Layout(display='none') # Oculto por defecto
)

layout_tab7 = widgets.VBox([
    widgets.HTML("<h2>7. Dimensionamiento de Planta de Emergencia</h2>"),
    widgets.VBox([
        emerg_load_label,
        widgets.HTML("<b>Consumidores alimentados en Emergencia:</b>"),
        emerg_consumidores_output
    ], layout=widgets.Layout(border='1px solid #ccc', padding='10px', margin='10px 0')),
    widgets.VBox([
        emerg_source_select, emerg_gen_box, emerg_bat_box
    ], layout=widgets.Layout(border='1px solid #ccc', padding='10px', margin='10px 0'))
])

# --- Pestaña 8: Diagrama Unifilar ---
diagrama_output = widgets.Output(layout=widgets.Layout(border='1px solid #ccc', padding='10px', margin='10px 0', min_height='600px', overflow='auto'))
boton_generar_diagrama = widgets.Button(description="Generar/Actualizar Diagrama", button_style='primary', icon='project-diagram')
boton_exportar_diagrama = widgets.Button(description="Exportar Diagrama (SVG)", button_style='info', icon='download', layout=widgets.Layout(margin='5px'))
diagrama_export_output = widgets.Output()

layout_tab8 = widgets.VBox([
    widgets.HTML("<h2>8. Diagrama Unifilar (SLD)</h2>"),
    widgets.HBox([boton_generar_diagrama, boton_exportar_diagrama]),
    diagrama_export_output,
    diagrama_output
])


# --- 4. CREACIÓN DE LAS PESTAÑAS (TABS) ---
app_tabs = widgets.Tab()
app_tabs.children = [layout_tab1, layout_tab2, layout_tab3, layout_tab4, layout_tab5, layout_tab6, layout_tab7, layout_tab8]
app_tabs.set_title(0, "1. Configuración")
app_tabs.set_title(1, "2. Balance Cargas")
app_tabs.set_title(2, "3. Dimensionamiento")
app_tabs.set_title(3, "4. Cálculos Aux.")
app_tabs.set_title(4, "5. Requisitos")
app_tabs.set_title(5, "6. Selección Gen.")
app_tabs.set_title(6, "7. Emergencia")
app_tabs.set_title(7, "8. Diagrama SLD")


# --- 5. LÓGICA DE LA APLICACIÓN (REFACTORIZADA) ---

def on_actualizar_calculos_clicked(b):
    """Función wrapper para el botón de actualizar."""
    with json_output:
        clear_output()
        display(widgets.HTML("<i><i class='fa fa-refresh fa-spin'></i> Actualizando todos los cálculos...</i>"))
        calcular_balance_y_dimensionamiento() # Ejecutar la función principal
        clear_output()
        display(widgets.HTML("<p style='color:green;'><b>¡Cálculos actualizados!</b> Revise las otras pestañas.</p>"))


def on_agregar_consumidor_clicked(b):
    """Agrega un nuevo consumidor (NUEVA LÓGICA)."""
    global global_consumidores

    # 1. Crear la estructura base del consumidor desde los inputs
    c_base = {
        'nombre': nuevo_nombre.value,
        'cantidad': nuevo_cantidad.value,
        'pn_kw_total': nuevo_pn_total.value,
        'cos_phi': nuevo_cos_phi.value,
        'esencial': nuevo_esencial.value,
        'barra': nuevo_barra.value,
        'modos': {}
    }
    for modo in modos_operacion:
        c_base['modos'][modo] = {
            'ku': nuevo_modos_w[modo]['ku'].value,
            'ks': nuevo_modos_w[modo]['ks'].value
        }

    # Convertir a (Ku, Ks) para el helper
    modos_ku_ks = {modo: (c_base['modos'][modo]['ku'], c_base['modos'][modo]['ks']) for modo in modos_operacion}

    # 2. Crear el diccionario completo (con widgets)
    nuevo_c_dict = get_consumidor_dict(get_consumidor_default(
        c_base['nombre'], c_base['cantidad'], c_base['pn_kw_total'], c_base['cos_phi'],
        c_base['esencial'], c_base['barra'], modos_ku_ks
    ))

    # 3. Agregarlo a la lista global
    global_consumidores.append(nuevo_c_dict)

    # 4. Actualizar la UI
    actualizar_lista_consumidores_ui()
    on_actualizar_calculos_clicked(None)

    # 5. Limpiar los campos de entrada
    nuevo_nombre.value = ""
    nuevo_cantidad.value = 1
    nuevo_pn_total.value = 10.0
    nuevo_cos_phi.value = 0.85
    nuevo_esencial.value = False
    nuevo_barra.value = "N/A"
    for modo in modos_operacion:
        nuevo_modos_w[modo]['ku'].value = 0.0
        nuevo_modos_w[modo]['ks'].value = 1.0 # Resetea a 1 (para n=1)

def on_quitar_consumidor_clicked(b):
    """Quita un consumidor de la lista."""
    global global_consumidores
    global_consumidores = [c for c in global_consumidores if c['id'] != b.consumidor_id]
    actualizar_lista_consumidores_ui()
    on_actualizar_calculos_clicked(None)

def actualizar_lista_consumidores_ui():
    """Limpia y reconstruye la lista de widgets de consumidores (Pestaña 1)"""
    items_para_vbox = []

    # --- 1. Encabezados (NUEVA LÓGICA) ---
    header_box = widgets.HBox([
        widgets.HTML("<b>Consumidor</b>", layout=widgets.Layout(width='30%')),
        widgets.HTML("<b>Cant.</b>", layout=widgets.Layout(width='8%')),
        widgets.HTML("<b>Pn Total (kW)</b>", layout=widgets.Layout(width='12%')),
        widgets.HTML("<b>Cos φ</b>", layout=widgets.Layout(width='10%')),
        widgets.HTML("<b>Esencial</b>", layout=widgets.Layout(width='12%')),
        widgets.HTML("<b>Barra (SLD)</b>", layout=widgets.Layout(width='15%')),
        widgets.HTML("<b>Acción</b>", layout=widgets.Layout(width='13%'))
    ], layout=widgets.Layout(width='100%', border='1px solid #aaa', padding='5px', background_color='#f0f0f0'))
    items_para_vbox.append(header_box)

    if not global_consumidores:
        items_para_vbox.append(widgets.HTML("<i>No hay consumidores.</i>"))

    # --- 2. Crear VBox para cada consumidor (NUEVA LÓGICA) ---
    for c_dict in global_consumidores:
        # --- 2a. Crear el HBox de Modos (Ku/Ks) ---
        modos_widgets_list = []
        for modo in modos_operacion:
            w_ku = c_dict['modos_widgets'][modo]['ku']
            w_ks = c_dict['modos_widgets'][modo]['ks']
            modos_widgets_list.append(widgets.VBox([
                widgets.HTML(f"<b>{modo}</b>"), w_ku, w_ks
            ], layout=widgets.Layout(width='19%', border='1px solid #f0f0f0', padding='2px', align_items='center')))
        modos_hbox = widgets.HBox(modos_widgets_list, layout=widgets.Layout(width='100%', justify_content='space-around'))

        # --- 2b. Crear el Acordeón ---
        modos_accordion = widgets.Accordion(children=[modos_hbox], selected_index=None)
        modos_accordion.set_title(0, "Ver/Ocultar Modos (Ku/Ks)")

        # --- 2c. Botón Quitar ---
        boton_quitar = widgets.Button(description="Quitar", icon='trash', button_style='danger', layout=widgets.Layout(width='95%'))
        boton_quitar.consumidor_id = c_dict['id']
        boton_quitar.on_click(on_quitar_consumidor_clicked)

        # --- 2d. Fila HBox principal (NUEVA LÓGICA) ---
        fila_principal = widgets.HBox([
            c_dict['nombre_w'],
            c_dict['qty_w'],      # <-- AÑADIDO
            c_dict['pn_w'],      # <-- AHORA ES PN TOTAL
            c_dict['cos_w'],
            c_dict['esencial_w'],
            c_dict['barra_w'],
            boton_quitar
        ], layout=widgets.Layout(width='100%', align_items='center'))

        # Asignar anchos
        fila_principal.children[0].layout.width = '30%' # Nombre
        fila_principal.children[1].layout.width = '8%'  # Cant
        fila_principal.children[2].layout.width = '12%' # Pn Total
        fila_principal.children[3].layout.width = '10%' # Cos
        fila_principal.children[4].layout.width = '12%' # Esencial
        fila_principal.children[5].layout.width = '15%' # Barra
        fila_principal.children[6].layout.width = '13%' # Botón

        # --- 2e. Crear el VBox final ---
        fila_consumidor_vbox = widgets.VBox([
            fila_principal, modos_accordion
        ], layout=widgets.Layout(border='1px solid #ddd', padding='5px', margin='5px 0'))
        items_para_vbox.append(fila_consumidor_vbox)

    # --- 3. Actualizar el VBox principal ---
    lista_consumidores_output.children = tuple(items_para_vbox)

def calcular_balance_y_dimensionamiento():
    """Función principal. Lee widgets, calcula balance y actualiza todas las pestañas."""

    # --- 1. Lectura de Datos (NUEVA LÓGICA) ---
    lista_consumo_real = [
        c for c in global_consumidores
        if ("generador" not in c['nombre_w'].value.lower())
    ]

    data_kw = []
    data_kva = []
    totales = {modo: {'kw': 0, 'kva': 0} for modo in modos_operacion}

    # --- Lógica para encontrar el motor más grande (NUEVA LÓGICA) ---
    motor_kw_max = 0
    motor_cos_phi = 0.8
    motor_nombre = "N/A"

    for c_dict in lista_consumo_real:
        pn_kw_total = c_dict['pn_w'].value
        cos_phi = c_dict['cos_w'].value
        cantidad = c_dict['qty_w'].value
        nombre = c_dict['nombre_w'].value

        # Actualizar motor más grande (lógica individual)
        pn_individual = (pn_kw_total / cantidad) if cantidad > 0 else pn_kw_total
        if pn_individual > motor_kw_max:
            motor_kw_max = pn_individual
            motor_cos_phi = cos_phi if cos_phi > 0 else 0.8
            motor_nombre = f"{nombre} ({motor_kw_max:.1f} kW)"

        row_kw = {"Consumidor": nombre, "Pn Total (kW)": pn_kw_total, "cos φ": cos_phi}
        row_kva = {"Consumidor": nombre, "Pn Total (kW)": pn_kw_total, "cos φ": cos_phi}

        for modo in modos_operacion:
            # --- FÓRMULA CENTRAL (NUEVA LÓGICA) ---
            ku = c_dict['modos_widgets'][modo]['ku'].value
            ks = c_dict['modos_widgets'][modo]['ks'].value
            pc_kw = pn_kw_total * ku * ks
            # --- FIN FÓRMULA ---

            sc_kva = pc_kw / cos_phi if cos_phi > 0 else 0

            row_kw[modo] = pc_kw
            row_kva[modo] = sc_kva
            totales[modo]['kw'] += pc_kw
            totales[modo]['kva'] += sc_kva

        data_kw.append(row_kw)
        data_kva.append(row_kva)

    # --- 2. Cálculo del Balance (Pestaña 2) ---
    with balance_output:
        clear_output(wait=True)
        if not data_kw:
            display(widgets.HTML("<p>No hay consumidores para calcular el balance.</p>"))
            # Limpiar Pestañas dependientes si no hay datos
            with sizing_output: clear_output(wait=True)
            verificar_requisitos(totales, "N/A", 0, 0) # Limpiar Pestaña 5
            actualizar_tab_seleccion_gen() # Limpiar Tab 6
            actualizar_tab_emergencia() # Limpiar Tab 7
            actualizar_tab_shore(0, 0) # Limpiar Tab 3
            motor_arranque_label.value = "Motor más grande: <b>N/A</b>"
            return

        df_kw = pd.DataFrame(data_kw)
        df_kva = pd.DataFrame(data_kva)
        total_kw_row = {"Consumidor": "TOTAL (kW)"}
        total_kva_row = {"Consumidor": "TOTAL (kVA)"}
        for modo in modos_operacion:
            total_kw_row[modo] = totales[modo]['kw']
            total_kva_row[modo] = totales[modo]['kva']

        df_kw = pd.concat([df_kw, pd.DataFrame([total_kw_row])], ignore_index=True)
        df_kva = pd.concat([df_kva, pd.DataFrame([total_kva_row])], ignore_index=True)

        display(widgets.HTML("<h3>Balance de Potencia Activa (kW)</h3>"))
        display(df_kw.style.format(precision=2).set_properties(**{'text-align': 'right'}))
        display(widgets.HTML("<hr><h3>Balance de Potencia Aparente (kVA)</h3>"))
        display(df_kva.style.format(precision=2).set_properties(**{'text-align': 'right'}))

    # --- 3. Cálculo de Dimensionamiento (Pestaña 3) ---
    with sizing_output:
        clear_output(wait=True)
        # Resetear estado global
        global_app_state.update({"max_kw": 0, "req_kw_n1_3gen": 0, "req_kw_n1_2gen": 0, "emerg_kw": 0, "emerg_kva": 0, "puerto_kw": 0, "puerto_kva": 0})

        modos_principales = [m for m in modos_operacion if m != "Emergencia"]
        peor_caso = "N/A"
        max_kw = 0
        max_kva = 0

        if any(totales[m]['kw'] > 0 for m in modos_principales):
            max_kw = max(totales[m]['kw'] for m in modos_principales)
            peor_caso = [m for m in modos_principales if totales[m]['kw'] == max_kw][0]
            max_kva = totales[peor_caso]['kva']

        if max_kw == 0:
            display(widgets.HTML("<p>La carga máxima calculada es 0 kW.</p>"))
            return

        # Guardar estado global
        global_app_state["max_kw"] = max_kw
        global_app_state["req_kw_n1_3gen"] = max_kw / 2
        global_app_state["req_kw_n1_2gen"] = max_kw
        global_app_state["emerg_kw"] = totales['Emergencia']['kw']
        global_app_state["emerg_kva"] = totales['Emergencia']['kva']
        global_app_state["puerto_kw"] = totales['Puerto']['kw']
        global_app_state["puerto_kva"] = totales['Puerto']['kva']

        display(widgets.HTML(f"<h4>Peor Caso (Planta Principal): {peor_caso}</h4>"))
        display(widgets.HTML(f"<ul><li>Potencia Activa Máxima: <b>{max_kw:.2f} kW</b></li><li>Potencia Aparente Máxima: <b>{max_kva:.2f} kVA</b></li></ul>"))
        display(widgets.HTML("<h4>Dimensionamiento N-1 (SOLAS)</h4>"))
        display(widgets.HTML(f"<b>Opción 3 Generadores (2 op.):</b> Se requieren 3 generadores de <b>{max_kw / 2:.2f} kW</b> cada uno."))
        display(widgets.HTML(f"<b>Opción 2 Generadores (1 op.):</b> Se requieren 2 generadores de <b>{max_kw:.2f} kW</b> cada uno."))

    # --- 4. Actualizar Pestañas Dependientes ---

    # Actualizar Pestaña 3 (Arranque Motor - NUEVA LÓGICA)
    global_app_state["largest_motor_kw"] = motor_kw_max
    global_app_state["largest_motor_cos_phi"] = motor_cos_phi
    motor_arranque_label.value = f"Motor más grande: <b>{motor_nombre}</b>"
    with arranque_output: clear_output()

    # Actualizar Pestaña 5 (Requisitos - NUEVA LÓGICA)
    verificar_requisitos(totales, peor_caso, max_kw, max_kva)

    # Actualizar Pestaña 2 (Gráfico)
    actualizar_grafico_balance(totales)

    # Actualizar Pestaña 3 (Toma de Tierra)
    actualizar_tab_shore(totales['Puerto']['kw'], totales['Puerto']['kva'])

    # Actualizar Pestaña 6 (Selección Generador)
    actualizar_tab_seleccion_gen()

    # Actualizar Pestaña 7 (Emergencia)
    actualizar_tab_emergencia()

def on_cargar_std_clicked(b):
    """Carga la lista de consumidores estándar."""
    global global_consumidores
    global_consumidores = get_lista_std()
    actualizar_lista_consumidores_ui()
    on_actualizar_calculos_clicked(None)
    with json_output: clear_output(); print("Lista estándar cargada.")

def on_limpiar_lista_clicked(b):
    """Borra todos los consumidores de la lista."""
    global global_consumidores
    global_consumidores = []
    actualizar_lista_consumidores_ui()
    on_actualizar_calculos_clicked(None)
    with trafo_output: clear_output()
    with balance_chart_output: clear_output()
    with shore_output: clear_output()
    with json_output: clear_output(); print("Lista de consumidores borrada.")

def on_calc_trafo_clicked(b):
    with trafo_output:
        clear_output(wait=True)
        carga_kva = input_carga_trafo_kva.value
        if carga_kva <= 0: print("Ingrese un valor positivo."); return
        req_kva = carga_kva * 1.15
        standar_sizes = [10, 15, 25, 30, 45, 50, 75, 100, 112.5, 150, 200, 225, 250, 300]
        sugerido = next((f"{size} kVA" for size in standar_sizes if size >= req_kva), "N/A")
        print(f"Carga Normal: {carga_kva:.2f} kVA | Requerida (115%): {req_kva:.2f} kVA | Recomendada: {sugerido}")

def on_calc_volt_drop_clicked(b):
    with volt_output:
        clear_output(wait=True)
        I, L_m, seccion_key, cos_phi, Vn = volt_I.value, volt_L.value, volt_seccion.value, volt_cosphi.value, volt_Vn.value
        if not all([I > 0, L_m > 0, Vn > 0, 0.1 <= cos_phi <= 1.0]):
            print("Error: Valores inválidos."); return
        cable_data = CABLE_DATABASE[seccion_key]
        R_ohm_km, X_ohm_km, ampacidad = cable_data['R_ohm_km'], cable_data['X_ohm_km'], cable_data['Ampacidad_A']

        print(f"--- 1. Verificación de Ampacidad ---")
        print(f"Corriente: {I:.2f} A | Ampacidad ({seccion_key}): {ampacidad:.2f} A")
        if I > ampacidad: print(f"ADVERTENCIA: ¡SUPERA AMPACIDAD!")
        else: print(f"OK: Ampacidad correcta.")

        print(f"\n--- 2. Verificación Caída de Tensión ---")
        R_ohm_m, X_ohm_m = R_ohm_km / 1000.0, X_ohm_km / 1000.0
        sin_phi = np.sin(np.arccos(cos_phi))
        delta_U_voltios = np.sqrt(3) * I * L_m * (R_ohm_m * cos_phi + X_ohm_m * sin_phi)
        delta_U_porc = (delta_U_voltios / Vn) * 100

        print(f"Caída de Tensión (ΔU): {delta_U_voltios:.2f} V ({delta_U_porc:.2f} %)")
        if delta_U_porc > 6.0: print(f"ADVERTENCIA: Supera el límite del 6%.")
        else: print(f"OK: Cae dentro del límite del 6%.")

def verificar_requisitos(totales, peor_caso, max_kw, max_kva):
    """Actualiza la Pestaña 5 (NUEVA LÓGICA)."""
    with requisitos_output:
        clear_output(wait=True)
        if max_kw == 0 and all(totales[m]['kw'] == 0 for m in totales):
            display(widgets.HTML("<p>No se ha calculado el balance.</p>")); return

        display(widgets.HTML("<h4>Consumidores Esenciales (SOLAS)</h4>"))
        lista_esenciales_html = "<ul>"
        count_esenciales = 0

        # Iterar sobre los widgets de la lista global
        for c_dict in global_consumidores:
            if c_dict['esencial_w'].value:
                ku = c_dict['modos_widgets']['Emergencia']['ku'].value
                ks = c_dict['modos_widgets']['Emergencia']['ks'].value
                pn_total = c_dict['pn_w'].value
                pc_kw = pn_total * ku * ks # Potencia en modo Emergencia

                if pc_kw > 0:
                    nombre = c_dict['nombre_w'].value
                    lista_esenciales_html += f"<li>{nombre} (<b>{pc_kw:.2f} kW</b>)</li>"
                    count_esenciales += 1

        if count_esenciales == 0: lista_esenciales_html += "<li>No hay consumidores esenciales activos en modo Emergencia.</li>"
        lista_esenciales_html += "</ul>"
        display(widgets.HTML(lista_esenciales_html))

        display(widgets.HTML("<hr><h4>Verificación Cumplimiento N-1</h4>"))
        if max_kw == 0: display(widgets.HTML("<p>No hay carga máxima calculada.</p>")); return

        display(widgets.HTML(f"<p>Carga máxima a cubrir (Peor caso: {peor_caso}): <b>{max_kw:.2f} kW</b></p>"))

        req_kw_3gen = global_app_state["req_kw_n1_3gen"]
        capacidad_3gen_n1 = req_kw_3gen * 2
        if capacidad_3gen_n1 >= max_kw: display(widgets.HTML(f"<p style='color:green; font-weight:bold;'>[OK] Opción 3 Gen. (c/u {req_kw_3gen:.2f} kW)</p>"))
        else: display(widgets.HTML(f"<p style='color:red; font-weight:bold;'>[FALLA] Opción 3 Gen. (c/u {req_kw_3gen:.2f} kW)</p>"))

        req_kw_2gen = global_app_state["req_kw_n1_2gen"]
        capacidad_2gen_n1 = req_kw_2gen * 1
        if capacidad_2gen_n1 >= max_kw: display(widgets.HTML(f"<p style='color:green; font-weight:bold;'>[OK] Opción 2 Gen. (c/u {req_kw_2gen:.2f} kW)</p>"))
        else: display(widgets.HTML(f"<p style='color:red; font-weight:bold;'>[FALLA] Opción 2 Gen. (c/u {req_kw_2gen:.2f} kW)</p>"))


# --- FUNCIONES PARA PESTAÑA 6 (GENERADOR PRINCIPAL) ---
def actualizar_tab_seleccion_gen():
    config = gen_config_select.value
    margen_porc = gen_margin_slider.value
    req_kw_base = global_app_state["req_kw_n1_3gen"] if config == '3gen' else global_app_state["req_kw_n1_2gen"]
    req_kw_final = req_kw_base * (1 + margen_porc / 100.0)
    gen_req_label.value = f"Potencia Requerida por Generador: <b>{req_kw_final:.2f} kW</b>"
    with gen_detalle_output: clear_output()
    gen_resultados_select.options = []
    global_app_state["selected_gen_kw"] = 0
    global_app_state["selected_gen_kva"] = 0
    icc_gen_kva_label.value = "Gen. kVA (de Pestaña 6): <b>--- kVA</b>"

def on_config_gen_changed(change):
    actualizar_tab_seleccion_gen()

def on_buscar_modelos_clicked(b):
    with gen_detalle_output: clear_output()
    config, margen_porc, frecuencia = gen_config_select.value, gen_margin_slider.value, gen_freq_select.value
    req_kw_base = global_app_state["req_kw_n1_3gen"] if config == '3gen' else global_app_state["req_kw_n1_2gen"]
    req_kw_final = req_kw_base * (1 + margen_porc / 100.0)
    if req_kw_final == 0:
        gen_resultados_select.options = [("N/A - Calcule el balance primero", "N/A")]; return

    resultados = []
    for fab, mod, p_60, p_50 in GEN_DATABASE:
        p_nominal = p_60 if frecuencia == 60 else p_50
        if p_nominal >= req_kw_final:
            label = f"{fab} {mod} ({p_nominal} kW @ {frecuencia}Hz)"
            unique_id = f"{fab}|{mod}|{p_nominal}|{frecuencia}"
            resultados.append((label, unique_id))

    if not resultados:
        gen_resultados_select.options = [("No se encontraron modelos", "N/A")]
    else:
        resultados_sorted = sorted(resultados, key=lambda x: float(x[1].split('|')[2]))
        gen_resultados_select.options = resultados_sorted

def on_gen_seleccionado_changed(change):
    with gen_detalle_output:
        clear_output(wait=True)
        selected_id = change['new']
        if not selected_id or selected_id == "N/A":
            global_app_state.update({"selected_gen_kw": 0, "selected_gen_kva": 0})
            icc_gen_kva_label.value = "Gen. kVA (de Pestaña 6): <b>--- kVA</b>"
            return
        try:
            fab, mod, p_nominal, frec = selected_id.split('|')
            p_nominal_float = float(p_nominal)
            p_nominal_kva = p_nominal_float / 0.8
            global_app_state.update({"selected_gen_kw": p_nominal_float, "selected_gen_kva": p_nominal_kva})
            icc_gen_kva_label.value = f"Gen. kVA (de Pestaña 6): <b>{p_nominal_kva:.2f} kVA</b>"
            display(widgets.HTML(f"<h4>Detalles del Modelo Seleccionado</h4><ul><li><b>Fabricante:</b> {fab}</li><li><b>Modelo:</b> {mod}</li><li><b>Potencia Nominal:</b> {p_nominal} kW ({p_nominal_kva:.2f} kVA @ 0.8 PF)</li></ul>"))
        except Exception as e:
            print(f"Error: {e}")
            global_app_state.update({"selected_gen_kw": 0, "selected_gen_kva": 0})
            icc_gen_kva_label.value = "Gen. kVA (de Pestaña 6): <b>--- kVA</b>"

# --- FUNCIONES PARA PESTAÑA 7 (EMERGENCIA) ---
def actualizar_tab_emergencia():
    emerg_kw = global_app_state["emerg_kw"]
    emerg_kva = global_app_state["emerg_kva"]
    emerg_load_label.value = f"Carga de Emergencia Calculada: <b>{emerg_kw:.2f} kW / {emerg_kva:.2f} kVA</b>"

    with emerg_consumidores_output:
        clear_output(wait=True)
        lista_html = "<ul>"
        count = 0
        for c_dict in global_consumidores:
            if c_dict['esencial_w'].value:
                ku = c_dict['modos_widgets']['Emergencia']['ku'].value
                ks = c_dict['modos_widgets']['Emergencia']['ks'].value
                pn_total = c_dict['pn_w'].value
                pc_kw = pn_total * ku * ks
                if pc_kw > 0 and "generador" not in c_dict['nombre_w'].value.lower():
                    lista_html += f"<li>{c_dict['nombre_w'].value} (<b>{pc_kw:.2f} kW</b>)</li>"
                    count += 1
        if count == 0: lista_html = "<p><i>No hay consumidores esenciales activos en modo Emergencia.</i></p>"
        else: lista_html += "</ul>"
        display(widgets.HTML(lista_html))

    with emerg_gen_detalle_output: clear_output()
    emerg_gen_resultados_select.options = []
    with emerg_bat_output: clear_output()
    global_app_state["selected_emerg_gen_kw"] = 0

def on_buscar_gen_emerg_clicked(b):
    with emerg_gen_detalle_output: clear_output()
    frecuencia, margen_porc = emerg_gen_freq_select.value, emerg_gen_margin_slider.value
    req_kw_base = global_app_state["emerg_kw"]
    req_kw_final = req_kw_base * (1 + margen_porc / 100.0)
    if req_kw_final == 0:
        emerg_gen_resultados_select.options = [("N/A - Carga de Emergencia es 0", "N/A")]; return

    resultados = []
    for fab, mod, p_60, p_50 in EMERGENCY_GEN_DATABASE:
        p_nominal = p_60 if frecuencia == 60 else p_50
        if p_nominal >= req_kw_final:
            label = f"{fab} {mod} ({p_nominal} kW @ {frecuencia}Hz)"
            unique_id = f"{fab}|{mod}|{p_nominal}|{frecuencia}"
            resultados.append((label, unique_id))

    if not resultados:
        emerg_gen_resultados_select.options = [("No se encontraron modelos", "N/A")]
    else:
        resultados_sorted = sorted(resultados, key=lambda x: float(x[1].split('|')[2]))
        emerg_gen_resultados_select.options = resultados_sorted

def on_gen_emerg_seleccionado_changed(change):
    with emerg_gen_detalle_output:
        clear_output(wait=True)
        selected_id = change['new']
        if not selected_id or selected_id == "N/A":
            global_app_state["selected_emerg_gen_kw"] = 0; return
        try:
            fab, mod, p_nominal, frec = selected_id.split('|')
            p_nominal_float = float(p_nominal)
            global_app_state["selected_emerg_gen_kw"] = p_nominal_float
            display(widgets.HTML(f"<h4>Detalles: {fab} {mod} ({p_nominal} kW)</h4>"))
        except Exception as e:
            print(f"Error: {e}")
            global_app_state["selected_emerg_gen_kw"] = 0

def on_emerg_source_changed(change):
    source = change['new']
    emerg_gen_box.layout.display = 'block' if source == 'Generador' else 'none'
    emerg_bat_box.layout.display = 'block' if source == 'Baterías' else 'none'

def on_calc_bateria_clicked(b):
    with emerg_bat_output:
        clear_output(wait=True)
        emerg_kw, horas, voltaje = global_app_state["emerg_kw"], emerg_bat_horas.value, emerg_bat_voltaje.value
        if emerg_kw <= 0: print("Error: Carga de emergencia es 0."); return
        if horas <= 0 or voltaje <= 0: print("Error: Horas y voltaje deben ser positivos."); return
        corriente_total_A = (emerg_kw * 1000) / voltaje
        capacidad_Ah = corriente_total_A * horas
        capacidad_req_Ah = capacidad_Ah / 0.8 # 80% DOD
        display(widgets.HTML(f"<h4>Dimensionamiento Banco de Baterías</h4>"))
        display(widgets.HTML(f"<p>Carga: {emerg_kw:.2f} kW | Tensión: {voltaje} V | Autonomía: {horas} h</p>"))
        display(widgets.HTML(f"Corriente: <b>{corriente_total_A:.2f} A</b>"))
        display(widgets.HTML(f"Capacidad Req. (80% DOD): <b>{capacidad_req_Ah:.2f} Ah</b>"))

# --- GUARDAR/CARGAR JSON (NUEVA LÓGICA) ---
def on_guardar_json_clicked(b):
    with json_output:
        clear_output(wait=True)
        config_data = []
        for c_dict in global_consumidores:
            consumidor = {
                'nombre': c_dict['nombre_w'].value,
                'cantidad': c_dict['qty_w'].value,         # <-- NUEVO
                'pn_kw_total': c_dict['pn_w'].value,     # <-- NUEVO
                'cos_phi': c_dict['cos_w'].value,
                'esencial': c_dict['esencial_w'].value,
                'barra': c_dict['barra_w'].value,
                'modos': {}
            }
            for modo in modos_operacion:
                consumidor['modos'][modo] = {
                    'ku': c_dict['modos_widgets'][modo]['ku'].value, # <-- NUEVO
                    'ks': c_dict['modos_widgets'][modo]['ks'].value  # <-- NUEVO
                }
            config_data.append(consumidor)

        json_string = json.dumps(config_data, indent=2)
        try:
            b64_json = b64encode(json_string.encode()).decode()
            href = f'<a href="data:application/json;base64,{b64_json}" download="balance_electrico_config_v2.json">Descargar Configuración (balance_electrico_config_v2.json)</a>'
            display(HTML(href))
        except Exception as e:
            print(f"Error al descargar: {e}")

def on_upload_json_changed(change):
    global global_consumidores
    with json_output:
        clear_output(wait=True)
        if not change['new']: return
        uploaded_file = change['new'][0]
        try:
            content_str = uploaded_file['content'].decode('utf-8')
            config_data = json.loads(content_str)
            nueva_lista_consumidores = []
            for c_base in config_data:
                # Compatibilidad con lógica nueva (y un poco de la vieja por si acaso)
                cantidad = c_base.get('cantidad', 1)
                pn_kw_total = c_base.get('pn_kw_total', c_base.get('pn_kw', 0)) # Pn total

                # Default modos
                modos_ku_ks = {}
                modos_data = c_base.get('modos', {})
                for modo in modos_operacion:
                    ku = modos_data.get(modo, {}).get('ku', modos_data.get(modo, {}).get('ksr', 0.0)) # Ku o Ksr
                    ks = modos_data.get(modo, {}).get('ks', modos_data.get(modo, {}).get('kn', 0.0))  # Ks o Kn
                    modos_ku_ks[modo] = (ku, ks)

                c_base_safe = {
                    'nombre': c_base.get('nombre', 'Sin Nombre'),
                    'cantidad': cantidad,
                    'pn_kw_total': pn_kw_total,
                    'cos_phi': c_base.get('cos_phi', 0.8),
                    'esencial': c_base.get('esencial', False),
                    'barra': c_base.get('barra', 'N/A'),
                    'modos_ku_ks': modos_ku_ks
                }
                nueva_lista_consumidores.append(get_consumidor_dict(get_consumidor_default(
                    c_base_safe['nombre'], c_base_safe['cantidad'], c_base_safe['pn_kw_total'],
                    c_base_safe['cos_phi'], c_base_safe['esencial'], c_base_safe['barra'], c_base_safe['modos_ku_ks']
                )))

            global_consumidores = nueva_lista_consumidores
            actualizar_lista_consumidores_ui()
            on_actualizar_calculos_clicked(None)
            print(f"Configuración cargada desde '{uploaded_file['name']}'.")
        except Exception as e:
            print(f"Error al leer JSON: {e}")
        upload_json.value = []


# --- OTRAS FUNCIONES (sin cambios) ---
def on_check_shore_clicked(b):
    with shore_output:
        clear_output(wait=True)
        carga_puerto_kva = global_app_state["puerto_kva"]
        capacidad_shore_kva = input_shore_kva.value
        print(f"Carga 'Puerto': {carga_puerto_kva:.2f} kVA | Capacidad Toma: {capacidad_shore_kva:.2f} kVA")
        if capacidad_shore_kva >= carga_puerto_kva: print(f"OK: La toma de tierra es SUFICIENTE.")
        else: print(f"ADVERTENCIA: La toma de tierra NO es suficiente. Faltan {carga_puerto_kva - capacidad_shore_kva:.2f} kVA.")

def on_calc_icc_clicked(b):
    with icc_output:
        clear_output(wait=True)
        gen_kva, gen_xd, Vn = global_app_state["selected_gen_kva"], icc_gen_xd.value, icc_Vn.value
        if gen_kva <= 0: print("Error: Seleccione un generador (Pestaña 6)."); return
        if not (0 < gen_xd < 1.0): print("Error: Xd'' debe ser p.u. (ej. 0.15)."); return
        if Vn <= 0: print("Error: Tensión inválida."); return
        In_A = (gen_kva * 1000) / (np.sqrt(3) * Vn)
        I_cc_A = In_A / gen_xd
        I_cc_kA = I_cc_A / 1000.0
        display(widgets.HTML(f"Gen: {gen_kva:.2f} kVA, In: <b>{In_A:.2f} A</b>"))
        display(widgets.HTML(f"Icc (en bornes): <b>{I_cc_kA:.2f} kA</b>"))

def on_verificar_arranque_clicked(b):
    with arranque_output:
        clear_output(wait=True)
        gen_kva = global_app_state.get("selected_gen_kva", 0)
        gen_xd_pu = gen_xd_transient.value
        motor_kw = global_app_state.get("largest_motor_kw", 0)
        motor_cos_phi = global_app_state.get("largest_motor_cos_phi", 0.8)
        lrc_factor = motor_lrc_factor.value
        cos_phi_arranque = motor_cos_phi_arranque.value

        if gen_kva == 0 or motor_kw == 0:
            print("Error: Calcule el balance Y seleccione un generador (Pestaña 6)."); return

        display(widgets.HTML("<h4>1. KVA de Arranque del Motor</h4>"))
        motor_kva = motor_kw / motor_cos_phi if motor_cos_phi > 0 else motor_kw / 0.8
        motor_in = (motor_kva * 1000) / (np.sqrt(3) * 440)
        kva_arranque = (np.sqrt(3) * 440 * (motor_in * lrc_factor)) / 1000.0
        display(widgets.HTML(f"Motor Pn: {motor_kw:.2f} kW | KVA (arranque): <b>{kva_arranque:.2f} kVA</b>"))

        display(widgets.HTML("<hr><h4>2. Capacidad del Generador</h4>"))
        gen_cap_arranque_kva = (1 / gen_xd_pu) * gen_kva
        display(widgets.HTML(f"Gen KVA (nominal): <b>{gen_kva:.2f} kVA</b> | Gen KVA (cap. arranque): <b>{gen_cap_arranque_kva:.2f} kVA</b>"))

        display(widgets.HTML("<hr><h4>3. Verificación</h4>"))
        if gen_cap_arranque_kva < kva_arranque:
            display(widgets.HTML(f"<p style='color:red; font-weight:bold;'>[FALLA]</p>"))
        else:
            display(widgets.HTML(f"<p style='color:green; font-weight:bold;'>[OK] El generador puede suministrar los KVA.</p>"))

        volt_dip_porc = (kva_arranque / gen_cap_arranque_kva) * 100
        display(widgets.HTML(f"Caída de tensión (Dip) estimada: <b>{volt_dip_porc:.2f} %</b>"))
        if volt_dip_porc > 15.0: display(widgets.HTML(f"<p style='color:orange;'>PRECAUCIÓN: Caída de tensión supera el 15%.</p>"))
        else: display(widgets.HTML(f"OK: Caída de tensión aceptable."))

def actualizar_grafico_balance(totales):
    with balance_chart_output:
        clear_output(wait=True)
        modos, kw_values = list(totales.keys()), [totales[m]['kw'] for m in totales.keys()]
        if sum(kw_values) == 0: display(widgets.HTML("<p><i>No hay datos.</i></p>")); return
        fig, ax = plt.subplots(figsize=(10, 5))
        bars = ax.bar(modos, kw_values, color='skyblue')
        ax.set_ylabel('Potencia Activa (kW)'); ax.set_title('Balance de Cargas'); ax.grid(axis='y', linestyle='--', alpha=0.7)
        ax.bar_label(bars, fmt='%.1f kW', padding=3)
        ax.set_ylim(top=max(kw_values) * 1.15)
        plt.tight_layout(); plt.show()

def actualizar_tab_shore(puerto_kw, puerto_kva):
    label_shore_carga.value = f"Carga en 'Puerto' (Calculada): <b>{puerto_kva:.2f} kVA</b> ({puerto_kw:.2f} kW)"
    with shore_output: clear_output()

def on_exportar_excel_clicked(b):
    with balance_export_output:
        clear_output(wait=True)
        # Recalcular DFs para exportar (NUEVA LÓGICA)
        lista_consumo_real = [c for c in global_consumidores if ("generador" not in c['nombre_w'].value.lower())]
        if not lista_consumo_real: print("No hay datos."); return

        data_kw, data_kva = [], []
        totales = {modo: {'kw': 0, 'kva': 0} for modo in modos_operacion}
        for c_dict in lista_consumo_real:
            pn_total, cos_phi, nombre = c_dict['pn_w'].value, c_dict['cos_w'].value, c_dict['nombre_w'].value
            row_kw, row_kva = {"Consumidor": nombre, "Pn Total (kW)": pn_total, "cos φ": cos_phi}, {"Consumidor": nombre, "Pn Total (kW)": pn_total, "cos φ": cos_phi}
            for modo in modos_operacion:
                ku, ks = c_dict['modos_widgets'][modo]['ku'].value, c_dict['modos_widgets'][modo]['ks'].value
                pc_kw = pn_total * ku * ks
                sc_kva = pc_kw / cos_phi if cos_phi > 0 else 0
                row_kw[modo], row_kva[modo] = pc_kw, sc_kva
                totales[modo]['kw'] += pc_kw
                totales[modo]['kva'] += sc_kva
            data_kw.append(row_kw); data_kva.append(row_kva)

        total_kw_row, total_kva_row = {"Consumidor": "TOTAL (kW)"}, {"Consumidor": "TOTAL (kVA)"}
        for modo in modos_operacion: total_kw_row[modo], total_kva_row[modo] = totales[modo]['kw'], totales[modo]['kva']
        df_kw = pd.concat([pd.DataFrame(data_kw), pd.DataFrame([total_kw_row])], ignore_index=True)
        df_kva = pd.concat([pd.DataFrame(data_kva), pd.DataFrame([total_kva_row])], ignore_index=True)

        try:
            output_stream = io.BytesIO()
            with pd.ExcelWriter(output_stream, engine='openpyxl') as writer:
                df_kw.to_excel(writer, sheet_name='Balance_kW', index=False)
                df_kva.to_excel(writer, sheet_name='Balance_kVA', index=False)
            output_stream.seek(0)
            b64_excel = b64encode(output_stream.read()).decode()
            href = f'<a href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64_excel}" download="balance_electrico_TP3.xlsx">Descargar Balance (balance_electrico_TP3.xlsx)</a>'
            display(HTML(href))
        except Exception as e:
            print(f"Error al generar Excel: {e}")

# --- FUNCIONES DIAGRAMA (sin cambios) ---
def on_generar_diagrama_clicked(b):
    global global_app_state
    with diagrama_output:
        clear_output(wait=True)
        display(widgets.HTML("<i>Generando diagrama unifilar...</i>"))
        try:
            gen_kw = global_app_state.get("selected_gen_kw", 0)
            gen_kw_label = f"{gen_kw:.0f}kW" if gen_kw > 0 else "N/A kW"
            gen_config = gen_config_select.value
            emerg_gen_kw = global_app_state.get("selected_emerg_gen_kw", 0)
            if emerg_gen_kw == 0: emerg_gen_kw = global_app_state.get("emerg_kw", 0)
            emerg_gen_label = f"{emerg_gen_kw:.0f}kW"

            consumidores_por_barra = {barra: [] for barra in barras_opciones}
            for c_dict in global_consumidores:
                barra = c_dict['barra_w'].value
                if barra != "N/A" and "generador" not in c_dict['nombre_w'].value.lower():
                    # NUEVA LÓGICA: Pn es Pn_total
                    pn_kw_total = c_dict['pn_w'].value
                    cantidad = c_dict['qty_w'].value
                    pn_individual = (pn_kw_total / cantidad) if cantidad > 0 else pn_kw_total
                    label = f"{c_dict['nombre_w'].value}"
                    sub_label = f"({pn_individual:.0f}kW)" if cantidad == 1 else f"({cantidad}x {pn_individual:.0f}kW)"

                    consumidores_por_barra[barra].append({'label': label, 'sub_label': sub_label})

            def draw_busbar(x, y, width, label):
                return f'<rect x="{x}" y="{y}" width="{width}" height="10" fill="black" /><text x="{x - 10}" y="{y + 8}" font-size="12" text-anchor="end" font-family="Arial">{label}</text>'
            def draw_generator(x, y, label):
                label_parts = label.split('\n')
                tspan = f'<tspan x="0" dy="0">{label_parts[0]}</tspan>'
                if len(label_parts) > 1: tspan += f'<tspan x="0" dy="1.2em">{label_parts[1]}</tspan>'
                return f'<g transform="translate({x},{y})"><circle cx="0" cy="0" r="15" stroke="black" stroke-width="2" fill="none" /><text x="0" y="5" font-size="14" text-anchor="middle" font-family="Arial" font-weight="bold">G</text><line x1="0" y1="15" x2="0" y2="30" stroke="black" stroke-width="2" /><text x="0" y="-20" font-size="12" text-anchor="middle" font-family="Arial">{tspan}</text></g>'
            def draw_load_icon(x, y, label, sub_label, icon_type='load'):
                icon_html = ""
                if icon_type == 'motor':
                    icon_html = '<circle cx="0" cy="0" r="15" stroke="black" stroke-width="2" fill="none" /><text x="0" y="5" font-size="14" text-anchor="middle" font-family="Arial" font-weight="bold">M</text>'
                else: # load
                    icon_html = '<rect x="-15" y="-10" width="30" height="20" stroke="black" stroke-width="2" fill="none" />'

                # Auto-wrap para etiqueta
                tspan_html = f'<tspan x="0" dy="0">{sub_label}</tspan>' # Sub-label (kW) primero
                words = label.split()
                current_line = ""
                for word in words:
                    if len(current_line + " " + word) > 20: # Límite 20 chars
                        tspan_html += f'<tspan x="0" dy="1.2em">{current_line.strip()}</tspan>'
                        current_line = word
                    else:
                        current_line += f" {word}"
                tspan_html += f'<tspan x="0" dy="1.2em">{current_line.strip()}</tspan>'

                return f'<g transform="translate({x},{y})">{icon_html}<line x1="0" y1="-{15 if icon_type=="motor" else 10}" x2="0" y2="-30" stroke="black" stroke-width="2" /><text x="0" y="25" font-size="11" text-anchor="middle" font-family="Arial">{tspan_html}</text></g>'
            def draw_switch(x1, y1, x2, y2, label, open=False):
                mid_x, mid_y = (x1 + x2) / 2, (y1 + y2) / 2
                open_offset = 8 if open else 0
                return f'<line x1="{x1}" y1="{y1}" x2="{mid_x - 5}" y2="{mid_y - 5}" stroke="black" stroke-width="2" /><line x1="{mid_x - 5}" y1="{mid_y - 5 - open_offset}" x2="{mid_x + 5}" y2="{mid_y + 5}" stroke="black" stroke-width="2" /><line x1="{mid_x + 5}" y1="{mid_y + 5}" x2="{x2}" y2="{y2}" stroke="black" stroke-width="2" /><text x="{mid_x + 8}" y="{mid_y - 8}" font-size="12" font-family="Arial">{label}</text>'
            def draw_transformer(x, y, label):
                return f'<g transform="translate({x},{y})"><circle cx="0" cy="-10" r="10" stroke="black" stroke-width="2" fill="none" /><circle cx="0" cy="10" r="10" stroke="black" stroke-width="2" fill="none" /><line x1="0" y1="-20" x2="0" y2="-30" stroke="black" stroke-width="2" /><line x1="0" y1="20" x2="0" y2="30" stroke="black" stroke-width="2" /><text x="0" y="45" font-size="12" text-anchor="middle" font-family="Arial">{label}</text></g>'

            svg_parts = ['<svg width="1200" height="700" xmlns="http://www.w3.org/2000/svg" style="background-color:white; border:1px solid #ccc; font-family: Arial;">']
            MSB_A_Y, MSB_B_Y, EMSB_Y, T1_BUS_Y = 150, 150, 450, 450
            GEN_Y, LOAD_Y, EMSB_LOAD_Y, T1_LOAD_Y = 50, 280, 580, 580
            MSB_A_X, MSB_A_W = 100, 450; MSB_B_X, MSB_B_W = 650, 450
            EMSB_X, EMSB_W = 100, 450; T1_BUS_X, T1_BUS_W = 650, 450

            svg_parts.append(draw_busbar(MSB_A_X, MSB_A_Y, MSB_A_W, "MSB-A 440V"))
            svg_parts.append(draw_busbar(MSB_B_X, MSB_B_Y, MSB_B_W, "MSB-B 440V"))
            svg_parts.append(draw_busbar(EMSB_X, EMSB_Y, EMSB_W, "EMSB 440V"))
            svg_parts.append(draw_busbar(T1_BUS_X, T1_BUS_Y, T1_BUS_W, "220V (T1)"))
            svg_parts.append(draw_switch(MSB_A_X + MSB_A_W, MSB_A_Y + 5, MSB_B_X, MSB_B_Y + 5, "Bus-Tie"))

            if gen_config == '2gen':
                g1_x = MSB_A_X + 150; svg_parts.append(draw_generator(g1_x, GEN_Y, f"G-1\n{gen_kw_label}")); svg_parts.append(draw_switch(g1_x, GEN_Y + 30, g1_x, MSB_A_Y, "Q1"))
                g2_x = MSB_B_X + MSB_B_W - 150; svg_parts.append(draw_generator(g2_x, GEN_Y, f"G-2\n{gen_kw_label}")); svg_parts.append(draw_switch(g2_x, GEN_Y + 30, g2_x, MSB_B_Y, "Q2"))
            else:
                g1_x = MSB_A_X + 100; svg_parts.append(draw_generator(g1_x, GEN_Y, f"G-1\n{gen_kw_label}")); svg_parts.append(draw_switch(g1_x, GEN_Y + 30, g1_x, MSB_A_Y, "Q1"))
                g2_x = MSB_A_X + 250; svg_parts.append(draw_generator(g2_x, GEN_Y, f"G-2\n{gen_kw_label}")); svg_parts.append(draw_switch(g2_x, GEN_Y + 30, g2_x, MSB_A_Y, "Q2"))
                g3_x = MSB_B_X + 150; svg_parts.append(draw_generator(g3_x, GEN_Y, f"G-3\n{gen_kw_label}")); svg_parts.append(draw_switch(g3_x, GEN_Y + 30, g3_x, MSB_B_Y, "Q3"))

            feed_norm_x = EMSB_X + 100; svg_parts.append(f'<line x1="{feed_norm_x}" y1="{MSB_A_Y + 10}" x2="{feed_norm_x}" y2="{EMSB_Y - 20}" stroke="black" stroke-width="2" />'); svg_parts.append(draw_switch(feed_norm_x, EMSB_Y - 20, feed_norm_x, EMSB_Y, "Feed Normal"))
            ge_x = EMSB_X + 250; svg_parts.append(draw_generator(ge_x, EMSB_Y - 100, f"G-E\n{emerg_gen_label}")); svg_parts.append(draw_switch(ge_x, EMSB_Y - 70, ge_x, EMSB_Y, "Feed Emerg.", open=True))
            t1_x = T1_BUS_X + 150; svg_parts.append(draw_transformer(t1_x, T1_BUS_Y - 100, "T1 440/220V")); svg_parts.append(f'<line x1="{t1_x}" y1="{T1_BUS_Y - 70}" x2="{t1_x}" y2="{T1_BUS_Y}" stroke="black" stroke-width="2" />'); svg_parts.append(draw_switch(MSB_B_X + MSB_B_W - 100, MSB_B_Y + 10, t1_x, T1_BUS_Y - 130, "Q-T1"))

            def draw_cargas_svg(barra_x, barra_y, barra_w, cargas, load_y):
                svg_cargas = []; num_cargas = len(cargas)
                if num_cargas == 0: return svg_cargas
                spacing = max(100, barra_w / (num_cargas + 1)); total_width_needed = spacing * (num_cargas - 1)
                start_x = barra_x + (barra_w - total_width_needed) / 2
                for i, c in enumerate(cargas):
                    pos_x = start_x + i * spacing
                    nombre_lower = c['label'].lower()
                    icon = 'motor' if any(term in nombre_lower for term in ["motor", "bomba", "thruster", "compresor", "ventilador"]) else 'load'
                    svg_cargas.append(draw_load_icon(pos_x, load_y, c['label'], c['sub_label'], icon))
                    svg_cargas.append(draw_switch(pos_x, barra_y + 10, pos_x, load_y - 30, f"Q-{i+1}"))
                return svg_cargas

            svg_parts.extend(draw_cargas_svg(MSB_A_X, MSB_A_Y, MSB_A_W, consumidores_por_barra["MSB-A"], LOAD_Y))
            svg_parts.extend(draw_cargas_svg(MSB_B_X, MSB_B_Y, MSB_B_W, consumidores_por_barra["MSB-B"], LOAD_Y))
            svg_parts.extend(draw_cargas_svg(EMSB_X, EMSB_Y, EMSB_W, consumidores_por_barra["EMSB"], EMSB_LOAD_Y))
            svg_parts.extend(draw_cargas_svg(T1_BUS_X, T1_BUS_Y, T1_BUS_W, consumidores_por_barra["220V (T1)"], T1_LOAD_Y))

            svg_parts.append('</svg>')
            svg_string = "\n".join(svg_parts)
            global_app_state["ultimo_diagrama_svg"] = svg_string
            clear_output(wait=True)
            display(HTML(svg_string))
        except Exception as e:
            clear_output(wait=True)
            display(widgets.HTML(f"<h3 style='color:red;'>Error al generar SVG:</h3><pre>{e}</pre>"))

def on_exportar_diagrama_clicked(b):
    with diagrama_export_output:
        clear_output(wait=True)
        svg_data = global_app_state.get("ultimo_diagrama_svg", "")
        if not svg_data: print("Error: Genere un diagrama primero."); return
        try:
            b64_svg = b64encode(svg_data.encode('utf-8')).decode('utf-8')
            href = f'<a href="data:image/svg+xml;base64,{b64_svg}" download="diagrama_unifilar_TP3.svg">Descargar Diagrama (diagrama_unifilar_TP3.svg)</a>'
            display(HTML(href))
        except Exception as e:
            print(f"Error al descargar: {e}")

# --- 6. CONECTAR BOTONES A FUNCIONES ---
boton_agregar.on_click(on_agregar_consumidor_clicked)
boton_cargar_std.on_click(on_cargar_std_clicked)
boton_limpiar_lista.on_click(on_limpiar_lista_clicked)
boton_actualizar_calculos.on_click(on_actualizar_calculos_clicked)
boton_calc_trafo.on_click(on_calc_trafo_clicked)
boton_check_shore.on_click(on_check_shore_clicked)
boton_verificar_arranque.on_click(on_verificar_arranque_clicked)
volt_boton.on_click(on_calc_volt_drop_clicked)
boton_calc_icc.on_click(on_calc_icc_clicked)
boton_exportar_excel.on_click(on_exportar_excel_clicked)
gen_config_select.observe(on_config_gen_changed, names='value')
gen_margin_slider.observe(on_config_gen_changed, names='value')
gen_buscar_boton.on_click(on_buscar_modelos_clicked)
gen_resultados_select.observe(on_gen_seleccionado_changed, names='value')
emerg_source_select.observe(on_emerg_source_changed, names='value')
emerg_gen_buscar_boton.on_click(on_buscar_gen_emerg_clicked)
emerg_gen_resultados_select.observe(on_gen_emerg_seleccionado_changed, names='value')
emerg_bat_calc_boton.on_click(on_calc_bateria_clicked)
boton_guardar_json.on_click(on_guardar_json_clicked)
upload_json.observe(on_upload_json_changed, names='value')
boton_generar_diagrama.on_click(on_generar_diagrama_clicked)
boton_exportar_diagrama.on_click(on_exportar_diagrama_clicked)


# --- 7. MOSTRAR LA APLICACIÓN ---
display(HTML("<h1>Herramienta de Balance Eléctrico (TP3 - 73.15) - v2.0 Integrada</h1>"))
display(HTML("<p><i>Versión actualizada con lógica de coeficientes (Cantidad, Pn Total, Ku, Ks).</i></p>"))

try:
    display(app_tabs)
    # --- 8. EJECUTAR CÁLCULOS INICIALES ---
    actualizar_lista_consumidores_ui()
    on_actualizar_calculos_clicked(None)
except Exception as e:
    display(HTML(f"<h2 style='color:red;'>Error Inesperado al Iniciar la App</h2><pre>{e}</pre>"))

print("¡Aplicación de Balance Eléctrico (v2.0) cargada con éxito!")

Cargando librerías estándar...


¡Aplicación de Balance Eléctrico (v2.0) cargada con éxito!
